# 作業工程計画

0. ライブラリを読み込む
1. 過去データを読み込む
2. 過去データを整形する
3. 過去データを学習してモデルを作る 
4. 設問データを読み込む
5. 設問データをモデルに読み込ませ、予測値を出す
6. 予測値を設問データに追加する
7. 6のデータに、分野別最終ペースの最大値、最小値、平均値を追加する
8. データを出力し、納品する

## ライブラリ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

#ライブラリの読み込み
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  KFold
import warnings
warnings.filterwarnings('ignore')

### 村上さん pandas_tool

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

## 読み込み

### DecisionTreeRegressorメソッドの概要

|引数名|概要|デフォルト|
| :---- | :---- | :---- |
|criterion|不純度を測定する基準（平均二乗誤差、平均絶対誤差など）|‘mse’|
|splitter|条件探索アルゴリズムを選択するオプション（’best’と’rondom’が指定可能）|‘best’|
|max_depth|決定木のノード深さの制限値。ツリーが深くなりすぎて過学習の状態に陥った際は、このパラメータが正則化の役割を果たす。|None|
|min_samples_split|ノードを分割するために必要なサンプルの最小値|2|
|min_samples_leaf|1ノードの深さを作成するために必要となるデータ数の最小値。指定した値以上のデータ数を持たないノードは作られない。|1|
|min_weight_fraction_leaf|サンプルの重みを考慮した上でのmin_samples_leafに該当|0.0|
|max_features|ランダムに指定する説明変数の数(全ての説明変数がモデル学習に活用されるわけではなく、ランダムに割り振られる）|None|
|random_state|乱数シード|None|
|max_leaf_nodes|作成される決定木の葉の数を、指定した値以下に制御する|None|
|min_impurity_decrease|決定木の成長の早期停止するための閾値。不純度が指定の値より減少した場合、ノードを分岐し、不純度が指定の値より減少しなければ分岐を抑制。|0.0|
|ccp_alpha|ccp_alphaが大きいほどプルーニングされるノードの数が増加。プルーニングとは、精度低下をできるだけ抑えながら過剰な重みを排除するプロセスを指す。|0.0|


### 関数の定義

In [ ]:
"""
1単語を文字数5で数える
機械学習用データを準備する関数
df[0]:21夏1年生モデル：検証データ ＝ 21年夏1学年
df[1]:21夏2年生モデル：検証データ ＝ 21年夏2学年
df[2]:21夏3年生モデル：検証データ ＝ 21年夏3学年
df[3]:21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def make_mldata(df,str):
    """
    df:使用するデータ
    str:抽出する科目
    1.選択した科目を抽出
    2.one hot encoding
    3.説明変数、目的変数でデータフレームを分離
    4.訓練用、検証用にデータフレームを分離
        1.21夏1年生モデル：検証データ ＝ 21年夏1学年
        2.21夏2年生モデル：検証データ ＝ 21年夏2学年
        3.21夏3年生モデル：検証データ ＝ 21年夏3学年
        4.21夏学年混合モデル：検証データ ＝ 21年夏全学年
    5.説明変数から採点回を削除 
    """
    #0.NAN remove 追加した
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df.loc[:,'文字数5'] = df.loc[:,'文字数5'].round(0).astype(int)
    #1.科目を抽出し、科目と分野1の列を削除
    df = df[df['科目']== str].drop(columns = ["科目","分野1","文字数","解答言語"])#解答言語を考慮しない
    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = "最終ペース")#目的変数を除外した（説明変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回_夏','学年','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離 不恰好なのをどうにかしたい
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train_1 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1))]
    X_train_2 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2))]
    X_train_3 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3))]
    X_train_4 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21))]
    #訓練用目的変数
    y_train_1 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1))]
    y_train_2 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2))]
    y_train_3 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3))]
    y_train_4 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21))]

    """夏が検証用"""
    #訓練用説明変数
    X_test_1 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1)]
    X_test_2 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2)]
    X_test_3 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3)]
    X_test_4 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)]
    #訓練用目的変数
    y_test_1 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1)]
    y_test_2 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2)]
    y_test_3 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3)]
    y_test_4 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)]
   
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_xb = ["採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_y = ["採点回_夏","年度","学年"]

    #説明変数
    X_train_1 = X_train_1.drop(columns = rem_cols_x)
    X_train_2 = X_train_2.drop(columns = rem_cols_x)
    X_train_3 = X_train_3.drop(columns = rem_cols_x)
    X_train_4 = X_train_4.drop(columns = rem_cols_x)

    #出力用に企画ペースを残している
    X_test_1b = X_test_1.drop(columns = rem_cols_xb)
    X_test_2b = X_test_2.drop(columns = rem_cols_xb)
    X_test_3b = X_test_3.drop(columns = rem_cols_xb)
    X_test_4b = X_test_4.drop(columns = rem_cols_xb)    
    
    X_test_1 = X_test_1.drop(columns = rem_cols_x)
    X_test_2 = X_test_2.drop(columns = rem_cols_x)
    X_test_3 = X_test_3.drop(columns = rem_cols_x)
    X_test_4 = X_test_4.drop(columns = rem_cols_x)
         
    #目的変数
    y_train_1 = y_train_1.drop(columns = rem_cols_y)
    y_train_2 = y_train_2.drop(columns = rem_cols_y)
    y_train_3 = y_train_3.drop(columns = rem_cols_y)
    y_train_4 = y_train_4.drop(columns = rem_cols_y)

    y_test_1 = y_test_1.drop(columns = rem_cols_y)
    y_test_2 = y_test_2.drop(columns = rem_cols_y)
    y_test_3 = y_test_3.drop(columns = rem_cols_y)
    y_test_4 = y_test_4.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[]
    data1=[X_train_1,y_train_1,X_test_1,y_test_1,X_test_1b]#1年生モデルデータ なぜかdata1はリストとして入っていて、X_trainとかはdf。
    data2=[X_train_2,y_train_2,X_test_2,y_test_2,X_test_2b]#2年生モデルデータ
    data3=[X_train_3,y_train_3,X_test_3,y_test_3,X_test_3b]#3年生モデルデータ
    data4=[X_train_4,y_train_4,X_test_4,y_test_4,X_test_4b]#学年混合モデルデータ
    df_res = [data1, data2, data3, data4]

    return df_res

In [89]:
"""
1単語を文字数5で数える
機械学習用データを準備する関数
21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def make_train(df,kamoku = str):
    """
    df:使用するデータ
    str:抽出する科目
    1.選択した科目を抽出
    2.one hot encoding
    3.説明変数、目的変数でデータフレームを分離
    4.訓練用、検証用にデータフレームを分離
    5.説明変数から採点回を削除 
    """
    #0.NAN remove 追加した
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df.loc[:,'文字数5'] = df.loc[:,'文字数5'].round(0).astype(int)
    #1.科目を抽出し、科目と分野1の列を削除
    df = df[df['科目']== kamoku].drop(columns = ["科目","分野1","文字数","解答言語"])#解答言語を考慮せず、１単語5文字で変換。分野は2の方。
    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = "最終ペース")#目的変数を除外した（説明変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回_夏','学年','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21))]
    #訓練用目的変数
    y_train = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21))]

    """夏が検証用"""
    #訓練用説明変数
    X_test = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)]
    #訓練用目的変数
    y_test = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)]
   
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回_夏","採点回_秋","採点回_冬","年度"]
    # rem_cols_xb = ["採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_y = ["採点回_夏","年度","学年"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test]#学年混合モデルデータ

    return df_res

In [89]:
"""
1単語を文字数5で数える
機械学習用データを準備する関数
df[3]:21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def make_Xtest(df,kamoku = str):
    """
    df:使用するデータ
    str:抽出する科目
    1.選択した科目を抽出
    2.one hot encoding
    3.説明変数、目的変数でデータフレームを分離
    4.訓練用、検証用にデータフレームを分離
    5.説明変数から採点回を削除 
    """
    #0.NAN remove 追加した
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df = df.drop(columns = "採点回","年度")#目的変数を除外した（説明変数だけ含む）データフレーム
    #1.科目を抽出し、科目と分野1の列を削除
    df = df[df['科目']== kamoku].drop(columns = ["科目"])
    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    return df

In [8]:
"""
https://stackoverflow.com/questions/50607740/reverse-a-get-dummies-encoding-in-pandas
ダミー変数を元に戻す。エクセル出力用
"""

def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [73]:
"""
make_mldataのseriesを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
"""
def predict_model(df1,df2,a=2,b=1,c=None):#本番用
    X_train= df1[0]
    y_train= df1[1]
    X_test = df2
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
    
    #得た結果を学習データとマージしてデータフレームで返す
    df_res=[]
    df_res = undummify(X_test)  #企画ペースを入れるためにここをいじった。元はX_test
    df_res = df_res.rename(columns={"文字数5":"文字数"})
    df_res.loc[:,'AI想定ペース']= y_pred #上のデータに予測値をマージ
    return df_res

In [73]:
"""
make_mldataのseriesを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
"""
def predict_model_proto(df,a=2,b=1,c=None):#練習用
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
    
    #得た結果を学習データとマージしてデータフレームで返す
    df_res=[]
    df_res = undummify(X_test)  #企画ペースを入れるためにここをいじった。元はX_test
    df_res = df_res.rename(columns={"文字数5":"文字数"})
    df_res.loc[:,'AI想定ペース']= y_pred #上のデータに予測値をマージ
    return df_res

In [82]:
def make_past_summary(df = df_ml_filled,category =str):
    """
    categoryで指定した文字列で集計した
    """
    #0.　生データのNAを削除
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])

    #訓練用説明変数
    df = df[~((df['採点回'] == "夏")&(df['年度']==21))]

    group_df1 = df[[category,'最終ペース']].groupby(category).mean().rename(columns={"最終ペース":"平均値"})
    group_df1.reset_index(inplace=True)

    group_df2 = df[[category,'最終ペース']].groupby(category).max().rename(columns={"最終ペース":"最大値"})
    group_df2.reset_index(inplace=True)

    group_df3 = df[[category,'最終ペース']].groupby(category).min().rename(columns={"最終ペース":"最小値"})
    group_df3.reset_index(inplace=True)

    group_df4 = df[[category,'最終ペース']].groupby(category).median().rename(columns={"最終ペース":"中央値"})
    group_df4.reset_index(inplace=True)


    # merge
    group_df = pd.merge(group_df1,group_df2, on = [category],how = 'left')
    group_df = pd.merge(group_df,group_df3, on = [category],how = 'left')
    group_df = pd.merge(group_df,group_df4, on = [category],how = 'left')
    return group_df

## target encoding

In [10]:
#データ読み込み マークダウンにしてあるから必要ならコードセルにする
"""
data1を加工したcsvファイルを読み込む場合　
"""

name_csv = "crlea_bunya_dm_0613received_0613cleaned_filled_test.csv" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#データが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
df = pd.read_csv(filepath_or_buffer = path_file, sep=",",
                 usecols=['科目',
                          '分野名_修正v1',
                          '分野名_修正v2',
                          'ポイント採点',
                          '年度',
                          '採点回',
                          '学年',
                          '置換後のポイント数',
                          # '置換後の文字数',
                          '置換後の文字数5',
                          # '解答言語',
                          '置換後の配点',
                          '企画ペース',
                          '最終ペース'])
#列のリネーム
df = df.rename(columns={'分野名_修正v1':'分野1',
                        '分野名_修正v2':'分野2',
                        '置換後のポイント数':'ポイント数',
                        # '置換後の文字数':'文字数',#英語1単語も1文字として数えた
                        '置換後の文字数5':'文字数',#英語１単語を5文字とした。前の分析ではこっち。精度を比較する。
                        '置換後の配点':'配点'})
df_raw = df

df1 = df_raw
df1 = df1.dropna(subset=['ポイント数', '配点', '文字数'])
df1.isnull().sum()

科目          0
企画ペース     149
最終ペース       0
年度          0
採点回         0
学年          0
分野1         0
分野2         0
ポイント採点      0
文字数         0
ポイント数       0
配点          0
dtype: int64

In [15]:
"""
holdout検証用の学習データと検証データを分割
1単語を文字数5で数える
機械学習用データを準備する関数
21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def sep_train_test(df):
    #0.NAN remove 追加した
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    # df_X = df.drop(columns = "最終ペース")#目的変数を除外した（説明変数だけ含む）データフレーム
    # df_y = df.loc[:,['年度','採点回','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    train = df[~((df['採点回'] == "夏")&(df['年度']==21))]

    """夏が検証用"""
    #訓練用説明変数
    test = df[(df['採点回'] == "夏")&(df['年度']==21)]
   
    #5.不要な変数を削除
    rem_cols= ["企画ペース","採点回","年度"]

    train = train.drop(columns = rem_cols)#.reset_index(inplace=True)
    test = test.drop(columns = rem_cols)#.reset_index(inplace=True)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[train,test]#学年混合モデルデータ

    return df_res

In [16]:
#目的変数の割合が等しくなるようにデータセットを分割
df = df1
df2 = sep_train_test(df)
print("訓練用データは" + str(len(df2[0])))
print("テスト用データは" + str(len(df2[1])))

TypeError: object of type 'NoneType' has no len()

In [33]:
df2[0].head()

,科目,最終ペース,学年,分野1,分野2,ポイント採点,文字数,ポイント数,配点
0,国語,143.4,1,評論,評論,1,60,3,6
1,国語,449.8,1,評論,評論,0,40,3,5
2,国語,693.1,2,評論,評論,0,20,3,10
3,国語,118.8,2,評論,評論,0,70,3,8
4,国語,149.7,1,評論,評論,1,70,3,6


In [21]:
#Holdout Target Encodingの関数定義
#Kfoldの定義
df= df1
column = "分野1"
target = "最終ペース"
kf = KFold(n_splits = 3, shuffle = True, random_state = 0)
#仮の箱を用意
box = np.zeros(len(df))
#Nanで埋めておく
box[:] = np.nan
#繰り返しながらTarget Encodingを行う
for train_index, test_index in kf.split(df):
    #２分割　訓練データと検証データ
    train_cv = df.iloc[train_index]
    # print(train_cv)
    test_cv = df[column].iloc[test_index]# dataframe columnのtest_index行目
    # print(test_cv)
    #column別のtarget平均をtraimで計算
    mean = train_cv.groupby(column)[target].mean()
    #boxにcolumn別のtarget平均を入れる

    
mean

分野1
いろいろな式          271.453236
データの分析          255.561402
ベクトル            202.479027
三角関数            238.187761
二次関数            255.855063
内容説明            374.870704
古文              415.605457
和文英訳             89.146906
図形と方程式          205.809042
図形と計量           295.068118
図形の性質           226.375317
場合の数と確率         257.894166
小説              196.792118
平面上の曲線と複素数平面    122.250953
微分法・積分法         154.875283
指数関数・対数関数       177.285844
数と式             283.565098
数列              195.060909
数列の極限           143.206546
整数の性質           189.500000
漢文              414.567555
自由英作文            54.158907
英作文             464.375432
英文和訳            260.326921
評論              257.580017
随筆              158.970513
Name: 最終ペース, dtype: float64

In [17]:
#Holdout Target Encodingの関数定義
def Holdout_te(df,column,target):
    #Kfoldの定義
    kf = KFold(n_splits = 3, shuffle = True, random_state = 0)
    #仮の箱を用意
    box = np.zeros(len(df))
    #Nanで埋めておく
    box[:] = np.nan
    #繰り返しながらTarget Encodingを行う
    for train_index, test_index in kf.split(df):
        #２分割　訓練データと検証データ
        train_cv = df.iloc[train_index]
        # print(train_cv)
        test_cv = df[column].iloc[test_index]# dataframe columnのtest_index行目
        # print(test_cv)
        #column別のtarget平均をtraimで計算
        mean = train_cv.groupby(column)[target].mean()
        #boxにcolumn別のtarget平均を入れる
        for i,m in mean.iteritems():#column名とvalueが入っている
            for v in test_cv.index:#testの行のインデックスを一つずつ
                if test_cv[v] == i: #columnmの変数名が
                    box[v] = m
    #新たな列に挿入
    df[column + "_target"] = box
    return df

df=Holdout_te(df1, "分野1", "最終ペース")
df=Holdout_te(df1, "分野2", "最終ペース")
# df


IndexError: index 479 is out of bounds for axis 0 with size 479

In [20]:
#[IN]:
#Holdout Target Encodingの関数定義
def Holdout_te(df,column,target):
    #Kfoldの定義
    kf = KFold(n_splits = 3, shuffle = True, random_state = 2)
    #仮の箱を用意
    box = np.zeros(len(df))
    #Nanで埋めておく
    box[:] = np.nan
    #繰り返しながらTarget Encodingを行う
    for train_index, test_index in kf.split(df):
        #２分割　訓練データと検証データ
        train = df.iloc[train_index]
        print(train)
        test = df[column].iloc[test_index]
        # print(test)
        #column別のtarget平均をtraimで計算
        mean = train.groupby(column)[target].mean()
        #boxにcolumn別のtarget平均を入れる
        for i,m in mean.iteritems():#indexとvalueかな。enumerateと同じ感じ。
            for v in test.index:#testのインデックスを一つずつ
                if test[v] == i: box[v] = m
    #新たな列に挿入
    df[column + "_target"] = box
    return df
df=Holdout_te(df, "column1", "target")
df=Holdout_te(df, "column2", "target")
# df


  column1 column2  target  column1_target  column2_target
2       A       D       0        0.333333        0.250000
3       A       D       1        0.333333        0.250000
6       B       D       1        0.500000        0.333333
7       B       D       1        0.500000        0.250000
8       B       E       0        0.500000             NaN
9       B       E       1        0.500000             NaN
  column1 column2  target  column1_target  column2_target
0       A       C       1            0.50             NaN
1       A       D       0            0.50        0.750000
4       A       D       0            0.50        0.750000
5       B       D       0            0.75        0.750000
6       B       D       1            0.50        0.333333
8       B       E       0            0.50             NaN
9       B       E       1            0.50             NaN
  column1 column2  target  column1_target  column2_target
0       A       C       1        0.500000             NaN
1       A     

# 結果出力

In [87]:
"""
予測値を算出し、過去の集計データと共に出力
国語
"""

df1 = make_train(df_past,"国語")#20&21の過去データを入力
df2 = make_Xtest(df_setsumon,"国語")#22夏の設問データを入力
df3 = predict_model(df1,df2)#df1のデータで訓練、df2のデータで予測した値
df4 = make_past_summary(df_past,"分野2")#20&21の過去データを入力
df_res = pd.merge(df3,df4, on = ["分野2"],how = 'left')#予測値に過去の集計データをマージ
df_res =df_res.rename(columns={"分野2":"分野"})
df_res['科目']="国語"
df_res.head(50)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,ポイント採点,学年,ポイント数,文字数5,配点,分野2_古文,分野2_小説,分野2_漢文,分野2_評論,分野2_随筆
165,0,1,2,30,4,0,0,0,1,0
166,1,1,3,60,6,0,0,0,1,0
187,0,2,3,60,5,0,0,0,1,0
188,1,2,3,70,8,0,0,0,1,0
216,0,3,3,50,9,0,0,0,1,0


In [174]:
"""
Excelの書き出し 
"""

df = df_res

name_excel_output = "crlea_predict_June15.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig',index=False)

name_excel_output = "crlea_predict_June15.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

In [83]:
df = df_ml_filled
df1 = predict_model(make_mldata(df,"国語")[3])
df2 = make_past_summary(df,"分野2")
df_res = pd.merge(df1,df2, on = ["分野2"],how = 'left')
df_res.head(50)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,ポイント採点,学年,ポイント数,文字数,配点,分野2,AI想定ペース,平均値,最大値,最小値,中央値
0,0,1,2,30,4,評論,375.100000,258.680738,1042.000000,79.4,192.914205
1,1,1,3,60,6,評論,143.400000,258.680738,1042.000000,79.4,192.914205
2,0,2,3,60,5,評論,118.800000,258.680738,1042.000000,79.4,192.914205
3,1,2,3,70,8,評論,131.000000,258.680738,1042.000000,79.4,192.914205
4,0,3,3,50,9,評論,171.800000,258.680738,1042.000000,79.4,192.914205
5,0,3,3,90,14,評論,138.000000,258.680738,1042.000000,79.4,192.914205
6,0,3,3,90,13,評論,255.313191,258.680738,1042.000000,79.4,192.914205
7,0,3,3,90,12,評論,255.313191,258.680738,1042.000000,79.4,192.914205
8,1,1,3,60,6,小説,138.500000,185.280962,363.700000,100.5,174.350000
9,1,2,3,60,5,小説,125.800000,185.280962,363.700000,100.5,174.350000


In [134]:
"""欠損値を補完したデータの予測値"""
#予測値と統計値を各科目のモデルごとにエクセルシートに出力


kamoku=["国語","英語","数学"]#,"数学"は文字数がないため、dropnaで除外してしまっている。前処理で変更する必要あり。
model=["1年","2年","3年","学年混合"]

mss= 2#min sample split
msl= 1#min sample leaf
md=None #max depth
test_num="June3"

name_file = "DTs_{}_mss{}_msl{}_md{}.xlsx".format(str(test_num),str(mss),str(msl),str(md)) #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
# path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

# 入力データ
df = df_ml_filled
df1=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
df2=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
for i,n in enumerate(kamoku):
    for j,m in enumerate(model):
        df1[i][j] = test_model2(make_mldata(df,kamoku[i])[j],mss,msl,md)
        df2[i][j] = pandas_tool.summary(df1[i][j].loc[:,['最終ペース','企画ペース','AI想定ペース','誤差','AI乖離度','元の乖離度']])[0]
        
with pd.ExcelWriter('{}/{}'.format(path_folder,name_file)) as writer:
    for i,n in enumerate(kamoku):
        for j,m in enumerate(model):
            s_names=['{}_{}_values'.format(kamoku[i],model[j]),'{}_{}_stats'.format(kamoku[i],model[j])]
            df1[i][j].to_excel(writer, sheet_name='{}'.format(s_names[0]))
            df2[i][j].to_excel(writer, sheet_name='{}'.format(s_names[1]))
            
            
# =IFS(M2>=600,6,M2>=500,5,M2>=400,4,M2>=300,3,M2>=200,2,M2>=100,1,TRUE,0)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a cop